# PHYS 210, Solo Worksheet 09
Due Thursday, Oct 12, 2023 at the start of class on Canvas

## *9.0 Introduction*
In this solo worksheet you are going to look at the impact of the time step on the accuracy of Euler's method and then we will introduce an alternate algorithm to Euler's method, RK4 (Runge-Kutta 4th order). You will compare the accuracy of Euler's method and RK4 for different step sizes

## *9.1 Limitations of Euler's method for motion*
The code below is modified from Reading 09 to add the exact solution and a graph of the residuals (the difference between Euler's solution and the exact solution). There is a version provided in markdown that you should copy into the cell below it to modify.

**Task 9.1.1** Take some time to make sure you understand how list comprehension is used to build the lists `y_exact` and `residuals_euler`.

**Task 9.1.2** With `dt = 0.1` (seconds) below, you can see that the Euler's method solution is not very accurate. Investigate how small you need to make `dt` before you feel that the Euler's method solution becomes reasonably accurate. How much worse is the accuracy for a really big step size such as `dt = 0.25`?

**Your notes on your investigation of the impact of `dt` on the accuracy of the Euler's method solution:**
* ..
* ..

```python
### This code is embedded in markdown so you can copy it into the cell below

def euler(y, v, t, t_end, t_step):
    """Euler's method integrator
    Args:
    - y: initial height
    - v: initial y-velocity
    - t, t_end: start and end time
    - t_step: step size
    Returns: list of times and corresponding y values
    """
    t_list = [t]
    y_list = [y]
    v_list = [v]
    a = -9.81  # gravitational acceleration
    
    while t < t_end:
        y = y + t_step * v
        v = v + t_step * a
        t = t + t_step

        y_list.append(y)
        v_list.append(v)
        t_list.append(t)
        
    return t_list, y_list

## Main part of the program
import numpy as np
import matplotlib.pyplot as plt

y0, v0 = 0, 10  # Initial position and velocity
t0, tmax, dt = 0, 2, 0.1  # Start, end times and step size

# Euler solution
times, y_euler = euler(y0, v0, t0, tmax, dt)

# Exact solution at the same times as Euler
y_exact = [y0 + v0*t - 0.5*9.81*t**2 for t in times]

# Residuals
residuals_euler = [y_euler[i] - (y0 + v0*t - 0.5*9.81*t**2) for i,t in enumerate(times)]

# Create figure and subplots
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

# Main plot (Euler's method vs Exact solution)
ax[0].plot(times, y_exact, 'r--', label='Exact Solution' ) 
ax[0].plot(times, y_euler, 'b', label="Euler's method" )
ax[0].set_title("One-dimensional motion")
ax[0].set_xlabel("t (s)")
ax[0].set_ylabel("y (m)")
ax[0].legend()
ax[0].grid(True)

# Residuals subplot
ax[1].plot(times, residuals_euler, 'b', label="Euler's method" )
ax[1].set_xlabel("t (s)")
ax[1].set_ylabel("Residual (m)")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()
```

In [ ]:
# Copy the code from above and explore the impact of different values of 'dt'



## *9.2 RK4 (Runge-Kutta 4th order)*
Spend 5-10 minutes learning about RK4 (Runge-Kutta 4th order), a much more accurate way to solve ODEs than Euler's method. Try some combination of Wikipedia, internet searches, ChatGPT and watching a short video, such as https://www.youtube.com/watch?v=C_WsQeOjbV4

To make sense of how most of the resources talk about determing a weighted average for the slope, our slope related to how our position increases is simply the velocity,

$$y_{i+1} = y_{i} + t_{step}v_{i}.$$

So RK4 is effectively evaluating the velocity at the time corresponding to $y_i$ and then at time $t_{step}/2$ and $t_{step}$ later, making a weighted average of these to get a better estimate of the overall velocity to use as the slope since the velocity is actually changing throughout the time step. 

In practice, the values $k_1$ to $k_4$ are $\Delta y$ terms since they include both $v$ and $t_{step}$.

We also apply this same process to how the velocity changes, 

$$v_{i+1} = v_{i} + t_{step}*a_{i},$$

but since our acceleration ($a=-g$) is constant over time, the $k$ terms all end up being the same. We will leave these in the code below so that the code works better as a somewhat general solution.

**Task 9.2.1:** Update the code below to make the same type of plots as as from question 9.1 (y vs t and residuals vs t) for the RK4 solutions

**Task 9.2.2:** Investigate the impact of `dt` on the accuracy of the RK4 solution. Contrast these results with what you found for the Euler's method solution. *Important! The residuals graph will likely have an overall multiplication factor, such as `1e-15` printed at the top of the y-axis*.

**Your notes on your investigation of the impact of `dt` on the accuracy of the RK4 solution:**
* ..
* ..

```python
### This code is embedded in markdown so you can copy it into the cell below

def rk4(y, v, t, t_end, t_step):
    """Runge-Kutta 4th order method integrator
    Args:
    - y: initial height
    - v: initial y-velocity
    - t, t_end: start and end time
    - t_step: step size
    Returns: list of times and corresponding y values
    """
    t_list = [t]
    y_list = [y]
    v_list = [v]
    a = -9.81  # gravitational acceleration

    while t < t_end:
        k1_y = t_step * v
        k1_v = t_step * a

        k2_y = t_step * (v + 0.5 * k1_v)
        k2_v = t_step * a

        k3_y = t_step * (v + 0.5 * k2_v)
        k3_v = t_step * a

        k4_y = t_step * (v + k3_v)
        k4_v = t_step * a

        y = y + (k1_y + 2 * k2_y + 2 * k3_y + k4_y) / 6.0
        v = v + (k1_v + 2 * k2_v + 2 * k3_v + k4_v) / 6.0
        t = t + t_step

        y_list.append(y)
        v_list.append(v)
        t_list.append(t)

    return t_list, y_list

## Main part of the program
import numpy as np
import matplotlib.pyplot as plt

y0, v0 = 0, 10  # Initial position and velocity
t0, tmax, dt = 0, 2, 0.1  # Start, end times and step size

# RK4 solution
times, y_rk4 = rk4(y0, v0, t0, tmax, dt)

# Exact solution at the same times as RK4
y_exact = [y0 + v0*t - 0.5*9.81*t**2 for t in times]

# Residuals
residuals_rk4 = [y_rk4[i] - (y0 + v0*t - 0.5*9.81*t**2) for i,t in enumerate(times)]

## Your graphing code
```

In [ ]:
# Copy the code from above, update the graphs, and then use
# this code to explore the impact of different values of 'dt'





## *9.3 Use a function for the exact solution*
In these types of ODE solvers, we're going to start using functions to do calculations that show up multiple times or govern motion, such as our exact solution equation. 

**Task 9.3.1:** Copy and modify the code from below such that the value calculated in and returned from the function `exact_soln` makes it so everything is working the same as in the code originally provided for 9.3 above.

```python
### This code is embedded in markdown so you can copy it again as needed
### The next cell has the code for you to modify

def exact_soln(y0, v0, t):
    a = -9.81
    # Add some code to calculate y and return it

def euler(y, v, t, t_end, t_step):
    """Euler's method integrator
    Args:
    - y: initial height
    - v: initial y-velocity
    - t, t_end: start and end time
    - t_step: step size
    Returns: list of times and corresponding y values
    """
    t_list = [t]
    y_list = [y]
    v_list = [v]
    a = -9.81  # gravitational acceleration

    while t < t_end:
        y = y + t_step * v
        v = v + t_step * a
        t = t + t_step

        y_list.append(y)
        v_list.append(v)
        t_list.append(t)

    return t_list, y_list

## Main part of the program
import numpy as np
import matplotlib.pyplot as plt

y0, v0 = 0, 10  # Initial position and velocity
t0, tmax, dt = 0, 2, 0.1  # Start, end times and step size

# Euler solution
times, y_euler = euler(y0, v0, t0, tmax, dt)

# Exact solution at the same times as Euler
y_exact = [exact_soln(y0, v0, t) for t in times]

# Residuals
residuals_euler = [y_euler[i] - exact_soln(y0, v0, t) for i,t in enumerate(times)]

# Create figure and subplots
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

# Main plot (Euler's method vs Exact solution)
ax[0].plot(times, y_exact, 'r--', label='Exact Solution' ) 
ax[0].plot(times, y_euler, 'b', label="Euler's method" )
ax[0].set_title("One-dimensional motion")
ax[0].set_xlabel("t (s)")
ax[0].set_ylabel("y (m)")
ax[0].legend()
ax[0].grid(True)

# Residuals subplot
ax[1].plot(times, residuals_euler, 'b', label="Euler's method" )
ax[1].set_xlabel("t (s)")
ax[1].set_ylabel("Residual (m)")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()
```

In [ ]:
# Your code here



## *Completing this solo worksheet and submitting it to Canvas*
Before submitting your work, restart + rerun your entire notebook to make sure that everything runs correctly and without error.

To do this:
1. **Restart & Run All:** From the "Kernel" menu to the right of the "Cell" menu, select "Restart & Run All". This will restart the python Kernel, erasing all variables currently stored in memory so that when you "Run All" cells, you can ensure that if you were to run your notebook again on a later day, it would run as intended.
1. Look through the whole notebook and make sure there are no errors. Many questions have purposeful errors in the distributed version so make sure you have fixed them all such that "Restart & Run All" will run through the whole book and successfully print "The notebook ran without errors" at the end. If you have any trouble resolving the errors, please ask one of your classmates or ask us in class or on Piazza.

**Export notebook as HTML:** After you've executed and checked your notebook, choose: File => Save_and_Export_Notebook_As => HTML. This will download an HTML version of your notebook to your computer. This version is can not be executed or modified. You may need to disable any pop-up blockers to allow the file to be downloaded.

**Submit to Canvas:** Submit the html file that you just downloaded to the appropriate Solo Worksheet submission on Canvas.

In [ ]:
print("The notebook ran without errors")